<a href="https://colab.research.google.com/github/rabina302/StarredPaperMaster/blob/main/BioBERT_Tf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import torch
from transformers import AutoTokenizer, AutoModel
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Load your data
data = pd.read_csv('/content/drive/MyDrive/StarredPaper/Data/data.csv')
data = data.dropna(subset=['text', 'label']) # Remove all rows with missing values
data['text'] = data['text'].str.replace(r'[^\w\s]+', '', regex=True) # Remove all punctuation
data['text'] = data['text'].str.encode('ascii', 'ignore').str.decode('ascii') # Remove all non-ASCII chars

In [ ]:
texts = data['text']
labels = data['label']

In [ ]:
# Configuration
model_name = "dmis-lab/biobert-base-cased-v1.1"
max_length = 128

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

config.json:   0%|          | 0.00/313 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [ ]:
# Tokenization and Feature Extraction
def extract_document_embeddings(texts):
    input_ids = []
    attention_masks = []
    for text in texts:
        encoding = tokenizer(text, truncation=True, padding='max_length', max_length=max_length, return_tensors='pt')
        input_ids.append(encoding['input_ids'])
        attention_masks.append(encoding['attention_mask'])
    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_masks)
    document_embeddings = torch.mean(outputs.last_hidden_state, dim=1)
    return document_embeddings

document_embeddings = extract_document_embeddings(texts)

In [ ]:
document_embeddings

tensor([[ 0.0598, -0.1324, -0.1789,  ...,  0.1541,  0.1531,  0.0552],
        [-0.0254,  0.0948, -0.1939,  ...,  0.2965, -0.0279,  0.0145],
        [-0.0508,  0.0440, -0.2566,  ...,  0.3097,  0.0300,  0.0477],
        ...,
        [ 0.0441, -0.2717, -0.1959,  ...,  0.2461,  0.0964, -0.0085],
        [-0.0502, -0.2392, -0.1983,  ...,  0.2014,  0.2013, -0.0054],
        [-0.0691, -0.1303, -0.2642,  ...,  0.2274,  0.0408, -0.0258]])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(document_embeddings, labels, test_size=0.2, random_state=42)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense

In [ ]:
# Define the model architecture
model = Sequential()
model.add(LSTM(units=100, return_sequences=True, input_shape=(1, X_train.shape[1])))
model.add(LSTM(units=50, return_sequences=True))
model.add(LSTM(units=25))
model.add(Dropout(0.2))
model.add(Dense(units=10, activation='relu'))
model.add(Dense(units=1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 1, 100)            347600    
                                                                 
 lstm_1 (LSTM)               (None, 1, 50)             30200     
                                                                 
 lstm_2 (LSTM)               (None, 25)                7600      
                                                                 
 dropout_37 (Dropout)        (None, 25)                0         
                                                                 
 dense (Dense)               (None, 10)                260       
                                                                 
 dense_1 (Dense)             (None, 1)                 11        
                                                                 
Total params: 385671 (1.47 MB)
Trainable params: 385671 

In [ ]:
# Train the model
history = model.fit(X_train, y_train, epochs=100, batch_size=64, validation_data=(X_test, y_test))

TypeError: Cannot convert the argument `type_value`: torch.float32 to a TensorFlow DType.

In [ ]:
# Predict probabilities
y_pred_probs = model.predict(X_test_reshaped)

# Convert probabilities to class predictions
y_pred_classes = (y_pred_probs > 0.5).astype(int)
